In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import VGG16, Xception
import efficientnet.keras as efn
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions
import warnings
warnings.filterwarnings("ignore")

# Image parameters
image_size = (224, 224)
batch_size = 16

# Placeholder dataset paths for five clients
# Replace these with actual paths or generators for the respective clients
train_generators = [None] * 5  # Replace with actual generators for 5 clients' training data
validation_generators = [None] * 5  # Replace with actual generators for 5 clients' validation data

# Number of classes from training data (manually set, or derived from actual data)
num_output_classes = 2  # Set to the correct number of output classes for your task

# SimpleModel class definition
class TransferLearningModel:
    def build(self):
        base_model = efn.EfficientNetB7(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
        transfer_layer = base_model.get_layer('top_activation')
        conv_model = Model(inputs=base_model.input, outputs=transfer_layer.output)
        
        model = Sequential()
        model.add(conv_model)
        model.add(Flatten())
        model.add(Dense(1024, activation='relu'))
        model.add(Dense(num_output_classes, activation='softmax'))
        return model

# Learning rate schedule and optimizer
learning_rate_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.01,
    decay_steps=10000,
    decay_rate=0.9
)
optimizer = Adam(learning_rate=learning_rate_schedule)
loss_function = 'categorical_crossentropy'
evaluation_metrics = ['categorical_accuracy']

# Build and compile the global model (used for initializing local models in each round)
transfer_model = TransferLearningModel()
global_model = transfer_model.build()
global_model.compile(optimizer=optimizer, loss=loss_function, metrics=evaluation_metrics)

# Placeholder test data generator (replace with actual test data)
test_generator = None  # You need to create a test generator for actual data

# Initial global model prediction (requires actual test data)
# Uncomment this when you have a valid test generator
# global_model.predict(test_generator)

# Federated learning helper functions
def scale_model_weights(weights, scale_factor):
    return [scale_factor * weight for weight in weights]

def aggregate_scaled_weights(scaled_weights):
    return [tf.reduce_sum(weight, axis=0) for weight in zip(*scaled_weights)]

# Incremental learning for Federated learning process for 5 clients
num_comm_rounds = 60
for round_num in range(num_comm_rounds):
    global_weights = global_model.get_weights()  # Get global weights before each round
    scaled_weights_per_client = []

    for client_idx in range(5):  # Loop over 5 clients
        local_model = transfer_model.build()  # Rebuild the local model for each client
        local_model.compile(optimizer=optimizer, loss=loss_function, metrics=evaluation_metrics)
        local_model.set_weights(global_weights)  # Initialize with global model weights
        
        # Placeholder steps, replace with actual steps_per_epoch based on your data
        train_steps = 100  # Replace with actual steps per epoch for the client's data
        validation_steps = 20  # Replace with actual validation steps for the client's data

        # Incremental learning (train on new data and retain previously learned weights)
        local_model.fit(
            train_generators[client_idx],  # Training data for this client
            epochs=1,
            steps_per_epoch=train_steps,
            validation_data=validation_generators[client_idx],  # Validation data for this client
            validation_steps=validation_steps
        )

        # Scale and collect weights from this client
        scaled_weights = scale_model_weights(local_model.get_weights(), 1 / 5)  # 1/5 for 5 clients
        scaled_weights_per_client.append(scaled_weights)

    # Update global model weights with the averaged weights from 5 clients
    averaged_weights = aggregate_scaled_weights(scaled_weights_per_client)
    global_model.set_weights(averaged_weights)

    # Simulate evaluation on test data (requires actual test data)
    if test_generator is not None:
        test_scores = global_model.evaluate(test_generator)
        print(f"Communication round {round_num}: Accuracy = {test_scores[1] * 100:.2f}%")

# Final prediction after training (requires actual test data)
# Uncomment this when you have a valid test generator
# predictions = global_model.predict(test_generator)
